### Plotting PTMs in Comet and PEAKS results using python for *Prochlococcus* MED4 benchmark study

Starting with:

    Peaks de novo results (.csv) of PTM-optimized sequencing
    Comet results from PTM-optimized database searches

Goal:

    Figures comparing the modifcation type and extent in database and de novo peptides
    
Using:

    - pandas
    - matplotlib

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib import pyplot
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [22]:
# read in csvs made in MED4-PeaksDB and MED4-Comet notebooks

peaks = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/PeaksDN/RAL95_MED2_trypsin_combine_PTMopt_DN50_totals.csv")
comet = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/RAL4_95_MED2_trypsin/TPP/RAL95_MED2_trypsin_combine_Comet_totals.csv")

frames = [peaks, comet]
index = ['peaks50', 'comet3']

# concatenate dataframes
ptm = pd.concat(frames, sort=False, ignore_index=True)

ptm.reindex(index)

ptm.head()

,Unnamed: 0,A,C,D,E,F,G,H,I,K,...,k-iron,k-meth,r-meth,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ iron,% K w/ meth,% R w/ meth
0,sample total,26298,4530,16675,28228,13678,18885,6541,0,37569,...,3157,3887,6077,1.000000,0.437152,0.181737,0.032968,0.084032,0.103463,0.303774
1,sample total,131053,26238,126142,156032,87249,146863,27981,175266,210151,...,38003,32781,16426,0.928691,0.449524,0.338037,0.322984,0.180837,0.155988,0.195771
